In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as sla

%matplotlib inline
import matplotlib.pyplot as plt

#### Exercicio 1

In [10]:
def elimin_gaussiana(a,b,imp = False, norma='inf'):
     #matriz estendida Me
    b = np.mat(b).T
    Me = np.concatenate((a,b), axis=1)
    if imp: print('Matriz estendida: \n',Me,'\n\n')
    if imp: print('='*80)
    for j in range(Me.shape[-1] - 2): #coluna
        if imp: print('Operação na coluna', j)
        if imp: print('='*80, '\n\n')
        for i in range(j+1, Me.shape[0]): #linha
            if (Me[j,j] != 0):
                Me[i] = Me[i] - (Me[i,j]/Me[j,j])*Me[j] #operações com cada linha
            else:
                Me[i] = Me[i] - (0)*Me[j]
            if imp: print('Operação na linha', i, ':',Me[i], '\n')
            if imp: print('Matriz escalonando: \n',Me,'\n')
        if imp: print('='*80)
    if imp: print('\nMatriz escalonada: \n',Me,'\n')
    Matriz_A = np.delete(Me, -1, axis=1) #matriz de coeficientes escalonada
    Vetor_b = Me[:,-1] #matriz de termos independentes escalonada
    Vetor_x = sla.solve_triangular(Matriz_A, Vetor_b) # solução do sistema escalonado
    if imp: print('='*80,'\n\nSolução do sistema: \n', Vetor_x)
    #verificação do erro do sistema
    if norma is 'inf': norma = np.inf
    if norma is '-inf': norma = -np.inf
    Erro = sla.norm(b - a.dot(Vetor_x),ord = norma)
    return (Matriz_A, Vetor_b, Vetor_x, Erro)

In [11]:
a = np.array([[2,3,-1],[4,4,-3],[2,-3,1]])
b = np.array([5,3,-1])
M = elimin_gaussiana(a,b)
M

(matrix([[ 2,  3, -1],
         [ 0, -2, -1],
         [ 0,  0,  5]]), matrix([[ 5],
         [-7],
         [15]]), array([[1.],
        [2.],
        [3.]]), 0.0)

#### Exercicio 2

O erro pode ser medido por $||ax-b|| = erro$, uma vez que um sistema linear é sempre da seguinte forma $ax = b$, onde havendo solução (determinada ou não) $ax-b = 0$. O erro em questão ocorre pois o pivô passa a ser 0(causando uma divisão por 0).

#### Exercicio 3

In [22]:
A = np.array([[1,4,52],[27,110,-3],[22,2,14]])
b = np.array([57,134,38])
sla.solve(A,b)

array([1., 1., 1.])

#### Exercicio 4

In [21]:
def  elim_gaussiana2(A, b, pivotamento=None, imp=False, norma='inf'):
    b = np.mat(b).T
    me = np.array(np.concatenate((A, b), axis=1))
    n = len(me)

    if imp:
        print('Matriz estendida:\n{}\n'.format(me))
    if imp:
        print('=' * 80)

    # Eliminação
    for k in range(n - 1):
        if pivotamento:
            # Pivô
            maxindex = abs(me[k:, k]).argmax() + k
            if me[maxindex, k] == 0:
                raise ValueError("Matriz é singular.")
            # Trocas
            if maxindex != k:
                me[[k, maxindex]] = me[[maxindex, k]]

                if imp:
                    print("Troca entre as linhas {} e {}".format(k, maxindex))
                if imp:
                    print('Matriz escalonando:\n{}\n'.format(me))
        else:
            if me[k, k] == 0:
                raise ValueError("Pivô é zero. Tente com a variável \'pivoteamento\' como True.")

        for linha in range(k + 1, n):
            mult = me[linha, k] / me[k, k]
            me[linha, k:] = me[linha, k:] - mult * me[k, k:]
            if imp:
                print('Operação na linha {}: {}'.format(linha, me[linha]))
            if imp:
                print('Matriz escalonando: \n {} \n'.format(me))

    # Retrosubstituição

    if imp:
        print('Matriz escalonada:\n{}\n'.format(me))

    matriz_a = me[:, :-1]  # matriz de coeficientes escalonada
    vetor_b = me[:, -1]  # matriz de termos independentes escalonada
    vetor_x = sla.solve_triangular(matriz_a, vetor_b)  # solução do sistema escalonado

    if imp:
        print('=' * 80, '\n\nSolução do sistema: \n', vetor_x)

    # Verificação do erro do sistema
    if norma is 'inf':
        norma = np.inf
    if norma is '-inf':
        norma = -np.inf
    erro = sla.norm(b - A.dot(vetor_x), ord=norma)

    return matriz_a, vetor_b, vetor_x, erro

In [23]:
A= np.array([[1., 4, 52], [27, 110, -3], [22, 2, 14]])
b = np.array([57., 134, 38])
elim_gaussiana2(A, b, pivotamento=True)

(array([[ 27.        , 110.        ,  -3.        ],
        [  0.        , -87.62962963,  16.44444444],
        [  0.        ,   0.        ,  52.09721048]]),
 array([134.        , -71.18518519,  52.09721048]),
 array([1., 1., 1.]),
 173.0)

#### Exercicio 5

In [31]:
e = 10
A = np.array([[e,2],[1,e]])
b = np.array([4,3])
eList = []

resultPivotiamento = []
resultSemPivotiamento = []
for i in range(100):
    resultPivotiamento.append(elim_gaussiana2(A, b, pivotamento=True)[2])
    resultSemPivotiamento.append(elim_gaussiana2(A, b, pivotamento=False)[2])
    eList.append(e)
    e = 10**(-i)
    A= np.array([[e,2],[1,e]])
pd.DataFrame({'Pivoteamento=True':resultPivotiamento,'Pivoteamento=False':resultSemPivotiamento, "e":eList})

,Pivoteamento=True,Pivoteamento=False,e
0,"[0.3555555555555555, 0.2222222222222222]","[0.3555555555555555, 0.2222222222222222]",1.000000e+01
1,"[2.0, 1.0]","[2.0, 1.0]",1.000000e+00
2,"[2.814070351758794, 1.8592964824120604]","[2.814070351758793, 1.8592964824120604]",1.000000e-01
3,"[2.9801490074503727, 1.9850992549627482]","[2.9801490074504056, 1.985099254962748]",1.000000e-02
4,"[2.9980014990007495, 1.9985009992504994]","[2.9980014990007575, 1.9985009992504996]",1.000000e-03
5,"[2.9998000149990003, 1.99985000999925]","[2.9998000149955573, 1.9998500099992502]",1.000000e-04
6,"[2.999980000149999, 1.9999850000999992]","[2.999980000151225, 1.9999850000999992]",1.000000e-05
7,"[2.9999980000015, 1.999998500001]","[2.9999979997974435, 1.999998500001]",1.000000e-06
8,"[2.999999800000015, 1.99999985000001]","[2.999999799691011, 1.99999985000001]",1.000000e-07
9,"[2.99999998, 1.999999985]","[2.999999981767587, 1.999999985]",1.000000e-08


#### Exercicio 6

Os resultados são diferentes pois o método lu retorna as matrizes L e U separadas, permitindo outras aplicações desse resultado sem a necessidade de manipulações nas matrizes.

Já o método lu_factor() retorna ambas as matrizes L e U unidas em uma só. Ou seja, no formato ideal para ser usado como entrada para a função lu_solve.

#### Exercicio 7

Transformando os dados em matrizes e aplicando o algoritmo de solução.

In [34]:
mat_a = np.array([[225,0,-25,0], [0, 175, 0, -125], [225, 0, -275, 50], [0, 25, 250, -275]])
mat_b = np.array([1400,100, -2000, 0])
resultado = elim_gaussiana2(mat_a, mat_b, pivotamento=True)[2]
resultado

array([ 8.09983897, 12.35196687, 16.89855072, 16.49275362])